Возьмём комментарии к фильмам с https://huggingface.co/datasets/blinoff/kinopoisk

In [171]:
import pandas as pd
df = pd.read_json('kinopoisk.jsonl', lines=True)
df.sample(5)

,part,movie_name,review_id,author,date,title,grade3,grade10,content
22868,top250,Престиж (2006),4870,MC LOUD,2010-07-08,None,Good,10.0,\nЭто очень хорошее кино. Нет. Это просто заме...
3040,top250,Шерлок Холмс Игра теней (2011),28961,ТруБлад,2012-01-06,Holmes sweet Holmes,Bad,0.0,\nВступление Капитана Очевидность: В чем состо...
35287,bottom100,Приключения Алёнушки и Ерёмы (2008),25166,tulupoff mix,2012-08-05,FALL!,Bad,5.0,\n Что сказать по этому мультфильму? У на...
1693,top250,Самый быстрый «Индиан» (2005),18595,Juvenile,2007-09-04,Мечта. Цель. Возраст. Хопкинс.,Neutral,0.0,\nВот это надо видеть! Великий и ужасный Ганни...
5501,top250,Игры разума (2001),4507,little Julia,2011-01-09,Врачи ставят ему диагноз «параноидная шизофрен...,Good,9.0,"\nХм, четыре Оскара, четырнадцатое месть в Топ..."


Почистим данные и приведём их все к нижнему регистру, а также уберём предварительно пунткуацию 

In [172]:
df = df.drop(columns='part')
df = df.drop(columns='movie_name')
df = df.drop(columns='review_id')
df = df.drop(columns='author')
df = df.drop(columns='date')
df = df.drop(columns='title')
df = df.drop(columns='grade10')
df.sample(5)

,grade3,content
30305,Good,"\nКлассический вестерн, отточенный манерой и с..."
5051,Good,"\nФанатом фильмов Тима Бёртона я не являюсь, а..."
4164,Good,\n31 декабря прошлого года состоялась премьера...
23228,Neutral,\nЭто очередной фильм посмотренный мной в виде...
9671,Good,"\nПочти три часа. Три часа с декорациями, напо..."


In [173]:
df['text_clean'] = df['content'].replace(r'[^\w\s]',' ',regex=True).replace(r'\s+',' ',regex=True).str.lower()

In [174]:
df = df.drop(columns='content')
df.head()

,grade3,text_clean
0,Good,блеф одна из моих самых любимых комедий этот ...
1,Good,адриано челентано продолжает радовать нас сво...
2,Good,несомненно это один из великих фильмов 80 х г...
3,Good,эта фраза на мой взгляд отражает сюжет несомн...
4,Neutral,как пела земфира скорее всего по совершенно д...


Разделим комментарии на хорошие и плохие 

In [175]:
df_pos = df.loc[df['grade3'] == 'Good']
df_pos = df_pos.drop(columns='grade3')

In [176]:
df_neg = df.loc[df['grade3'] == 'Bad']
df_neg = df_neg.drop(columns='grade3')

Выделим только 100 положительных и отрицательных комментариев для составления словарей

In [177]:
pos_100 = df_pos['text_clean'].head(100)
neg_100 = df_neg['text_clean'].head(100)

Удалим стоп-слова и знаки препинания из двух словарей, а потом лемматизируем и токенизируем их

In [178]:
import re

from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    if len(tokens) > 2:
        return tokens
    return None

In [179]:
neg = neg_100.apply(lemmatize)

In [180]:
pos = pos_100.apply(lemmatize)

In [181]:
neg.head()

86     [уныло, вынудить, констатировать, славный, сту...
87     [собственно, говорить, экстраординарный, анима...
100    [классика, уолт, дисней, тот, мультфильм, кото...
192    [желание, посмотреть, мульт, рапунцель, возник...
203    [назвать, рапунцель, невменяемый, бред, свой, ...
Name: text_clean, dtype: object

In [182]:
pos_fin=[]
for i in pos:
    if i:
        pos_fin.append(i)
    else:
        break

In [183]:
neg_fin=[]
for i in neg:
    if i:
        neg_fin.append(i)
    else:
        break

Добавляем выборку 10 положительных и 10 отрицательных отзывов для проверки. Берём отзывы из конца таблицы, чтобы были отзывы, которые не учавствовали в создании словарей. Далее готовим данные к сравнению 

In [184]:
pos_10 = df_pos['text_clean'].tail(10)
neg_10 = df_neg['text_clean'].tail(10)

In [185]:
pos_10 = neg_10.apply(lemmatize)
neg_10 = neg_10.apply(lemmatize)

In [186]:
pos_10_fin = []
for i in pos_10:
    if i:
        pos_10_fin.append(i)
    else:
        break
        
neg_10_fin = []
for i in neg_10:
    if i:
        pos_10_fin.append(i)
    else:
        break

Оцениваем тональность

In [195]:
def get_tone(text, pos_fin, neg_fin):
    res = ''
    lemmas = lemmatize(text)
    s_pos = 0
    s_neg = 0
    for lem in lemmas:
        s_pos =+ pos_fin.count(lem)
        s_neg =+ neg_fin.count(lem)
    if s_neg == 0 or s_pos/s_neg > 1:
        res = 'Good'
    elif s_pos == 0 or s_neg/s_pos > 1:
        res = 'Bad'
    else:
        res = 'None'
    return(res)

In [196]:
get_tone('тупой и ещё тупее', pos_fin, neg_fin)

'Good'

Хорошим способом улучшить этот алгоритм будет присвоение веса каждому слову в отзыве, который нам надо оценить. Таким образом, если слово встречается только один раз, оно будет весить 1, а если слово встречаетя, например, три раза, оно будет весить 3.

И ещё один способ, который я хочу предложить для улучшения алгоритма - это расширение изначальной разметки. Например, датасет, с которым я изначально работала предоставлял возможность выделять нейтральные отзывы. Но ещё удобнее определять анализ тональности для русскоязычного текста с помощью специальной библиотеки Dostoevsky -  модели FastTextSocialNetworkModel. Таким образом мы бы получили на выходе пять категорий тональности: нейтральная тональность; негативная тональность; позитивная тональность; речь, в которой отсутствуют эмоционально окрашенные слова; отсутствие четко-выраженной тональности.